In [1]:
import pandas as pd
import numpy as np
import ast
from ast import literal_eval
import arrow
from sklearn.neighbors import NearestNeighbors
import joblib

/tmp/ipykernel_58661/2184389093.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv("songs_data.csv", sep=',', converters={'Artist Genres': lambda x: x.split(", ")}, keep_default_na=False)
df["Artist Genres"].head()

0        [acid house,ambient house,big beat,hip house]
1                        [dance pop,miami hip hop,pop]
2                                      [dance pop,pop]
3    [album rock,art rock,british invasion,classic ...
4      [album rock,british invasion,classic rock,rock]
Name: Artist Genres, dtype: object

In [3]:
#extract simpler genres
genres = ['pop', 'rock', 'hip hop', "house", "disco", "soul", "r&b"]
song_genres = []
for i in range(0, df.shape[0]):
    row = str(df["Artist Genres"].values[i][0])
    song_genres.append('')
    for g in genres:
        if row.find(g) != -1:
            song_genres[i] = g
            break

df["Genre"] = song_genres
df_all = pd.read_csv("songs_data.csv")
df_all["Genre"] = song_genres
df_all["Index"] = df_all.index

In [4]:
included_variables = ["Popularity", "Danceability","Energy","Loudness","Speechiness","Acousticness","Instrumentalness","Liveness","Tempo","Valence", "Album Release Date"]
indy_included = included_variables + ["Index"]
print(indy_included)

['Popularity', 'Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Tempo', 'Valence', 'Album Release Date', 'Index']


In [5]:
print(df_all["Album Release Date"])
df_all["Album Release Date"] = pd.to_datetime(df_all["Album Release Date"], format="mixed",  errors='coerce').view('int64') 
print(df_all["Album Release Date"])

0       1992-08-03
1       2009-10-23
2       1999-01-12
3       2014-10-20
4       1969-12-05
           ...    
9994    2022-06-17
9995    2005-10-24
9996    2000-08-14
9997    2023-01-06
9998    2023-05-19
Name: Album Release Date, Length: 9999, dtype: object
0        712800000000000000
1       1256256000000000000
2        916099200000000000
3       1413763200000000000
4         -2332800000000000
               ...         
9994    1655424000000000000
9995    1130112000000000000
9996     966211200000000000
9997    1672963200000000000
9998    1684454400000000000
Name: Album Release Date, Length: 9999, dtype: int64


/tmp/ipykernel_58661/2907213110.py:2: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df_all["Album Release Date"] = pd.to_datetime(df_all["Album Release Date"], format="mixed",  errors='coerce').view('int64')


In [6]:
def clean_and_norm(df_in):
    #normalize the data
    df_ints = pd.DataFrame(df_in, columns = indy_included)
    
    normed=(df_ints-df_ints.mean())/df_ints.std()
    normed=(df_ints-df_ints.min())/(df_ints.max()-df_ints.min())

    #clean data
    for column in indy_included:
        mean = float(normed[column].mean(skipna = True))
        normed[column] = normed[column].replace(np.NaN, mean)
    return normed

In [7]:
def knn_fit(name, data):
    #clean data
    normed = clean_and_norm(data)

    #grab normed data with only included variables
    train_data = pd.DataFrame(normed, columns = included_variables)
    train_data = train_data.iloc[1:,]
    
    knn = NearestNeighbors(metric='euclidean', algorithm='auto')
    knn.fit(train_data)

    song_i = int(data["Index"].iloc[0]) 
    test = pd.DataFrame(data, columns = included_variables)
    test = test.iloc[song_i:song_i+1:,]

    distances, indices = knn.kneighbors(test, n_neighbors=3)
    
    print(data["Track Name"].iloc[song_i], "by", data["Artist Name(s)"].iloc[song_i])
    print()
    print("Closest to:")
    
    for ind in indices[0]:
        print(data["Track Name"].iloc[ind + 1], "by", data["Artist Name(s)"].iloc[ind + 1])
    print()

    # save that model uh huh uh huh
    # load via: https://datascience.stackexchange.com/questions/52704/how-to-save-a-knn-model
    # joblib.dump(knn, name)

In [14]:
def move_knn(data, query, variable, direction):
    #clean data
    normed = clean_and_norm(data)

    #grab normed data with only included variables
    train_data = pd.DataFrame(normed, columns = included_variables)
    
    knn = NearestNeighbors(metric='euclidean', algorithm='auto')
    knn.fit(train_data)

    query_stripped = pd.DataFrame(normed, columns = included_variables)
    distances, indices = knn.kneighbors(query_stripped, n_neighbors=3)

    print("Input Song:")
    print(query["Track Name"], "by", query["Artist Name(s)"])

    # where are we going good sir
    direction_str = "more"
    if direction == 0: direction_str = "less"

    print("Our output has", direction_str, variable, "than our input:")
    ind = indices[0][0]
    print(data["Track Name"].iloc[ind + 1], "by", data["Artist Name(s)"].iloc[ind + 1])
    print()

In [12]:
def move_along(data, query_point, variable, direction):
    '''Data = input data (should be genre specific)
    query_point = current song
    variable = what are we changing? Acousticness? Loudness?
    direction = up/down (1 = up, 0 = down)'''
    filtered_data = data[data[variable].iloc[:,] > query_point[variable]]
    if direction == 0:    
        filtered_data = data[data[variable].iloc[:,] < query_point[variable]]
    
    # Ensure there is at least one point left after filtering
    if len(filtered_data) == 0:
        print("No neighbors found with a higher", variable,"value than the query point.")
    else:
        move_knn(filtered_data, query_point, variable, direction)

In [15]:
genre = "rock"
g_data = df_all[df_all['Genre'] == genre]

song_i = int(g_data["Index"].iloc[0]) 
query_point = g_data.iloc[song_i]

move_along(g_data, query_point, "Acousticness", 1)

Input Song:
Something About The Way You Look Tonight - Edit Version by Elton John
Our output has more Acousticness than our input:
You Can't Always Get What You Want by The Rolling Stones



In [ ]:
for genre in genres:
    print(genre)
    g_data = df_all[df_all['Genre'] == genre]
    name = genre + ".pkl"
    knn_fit(name, g_data)
    print("------------------------")